In [ ]:
# Load libraries

from pyspark.sql import SQLContext
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler
import utils
%matplotlib inline

In [ ]:
# Load minute weather data

sqlContext = SQLContext(sc)
df = spark.read.format("csv").option("header","true").option("inferSchema","true").load("file:<path>/minute_weather.csv") 

In [ ]:
# Count rows



In [ ]:
# Filter rows



In [ ]:
# Show summary statistics



In [ ]:
# Drop null data



In [ ]:
# Create feature vector

featuresUsed = ['air_pressure', 'air_temp', 'avg_wind_direction', 'avg_wind_speed', 'max_wind_direction', 
        'max_wind_speed','relative_humidity']
assembler = VectorAssembler(inputCols=featuresUsed, outputCol="features_unscaled")
assembled = assembler.transform(workingDF)

In [ ]:
# Scale data

scaler = StandardScaler(inputCol="features_unscaled", outputCol="features", withStd=True, withMean=True)
scalerModel = scaler.fit(assembled)
scaledData = scalerModel.transform(assembled)

In [ ]:
# Use one-third data for elbow plot

scaledData = scaledData.select("features", "rowID")

elbowset = scaledData.filter((scaledData.rowID % 3) == 0).select("features")
elbowset.persist()
elbowset.count()

In [ ]:
# Generate clusters for elbow plot



In [ ]:
# Show elbow plot



In [ ]:
# Run KMeans for k = 12

scaledDataFeat = scaledData.select("features")
scaledDataFeat.persist()

kmeans = KMeans(k=12, seed=1)
model = kmeans.fit(scaledDataFeat)
transformed = model.transform(scaledDataFeat)

In [ ]:
# Compute cluster centers

centers = model.clusterCenters()
P = utils.pd_centers(featuresUsed, centers)
centers

## Dry Days

In [ ]:
utils.parallel_plot(P[P['relative_humidity'] < -0.5], P)

## Humid Days

## Hot Days

## Cool Days